In [ ]:
%load_ext autoreload
%autoreload 2

import os,sys
import h5py  
import numpy as np
import math,cmath
from matplotlib.backends.backend_pdf import PdfPages
from scipy.optimize import curve_fit
import matplotlib as mpl
import matplotlib.pyplot as plt
mpl.style.use('default')
mpl.rcParams['figure.facecolor'] = 'white'
mpl.rcParams['figure.titlesize'] = 20
mpl.rcParams['figure.figsize'] = [6.4*1.2,4.8*1.2]
mpl.rcParams['axes.labelsize'] = 30
mpl.rcParams['axes.titlesize'] = 30
mpl.rcParams['lines.marker'] = 's'
mpl.rcParams['lines.linestyle'] = ''
mpl.rcParams['lines.markersize'] = 12
mpl.rcParams['errorbar.capsize'] = 12
mpl.rcParams['xtick.labelsize'] = mpl.rcParams['ytick.labelsize'] = 22
mpl.rcParams['legend.fontsize'] = 24

mpl.rcParams.update({"axes.grid" : True})
import util as yu

In [ ]:
path='/p/project/pines/li47/code/projectData/NST_b-discNJN/data_merge/NST_b.h5_main'
data=yu.load(path)
for ens in yu.ensembles:
    Ncfg=yu.deepKey(data[ens]['2pt'],2).shape[0]
    print(ens+': Ncfg='+str(Ncfg))

In [ ]:
def getdat(ens,ins,tfs,case):
    flags={
        'cc2pt':False,
        'cc3pt':False,
    }
    spins=[0,1]; flas=['N1','N2']; diags=['N','N_bw']
    # spins=[0,1]; flas=['N1']; diags=['N']
    diags_3pt=[diag+'-j' for diag in diags]
    t=np.mean([data[ens]['2pt']['t;0,0,0;{};{}_t;0,0,0;{};{}'.format(spin,fla,spin,fla)][diag] for spin in spins for fla in flas for diag in diags],axis=0)
    if flags['cc2pt']:
        t=(t+np.conj(t))/2
    t_2pta=t_2ptb=t
    
    cg,j=ins.split('_')
    gm={'gS':'id','gA':'g5gz','gT':'sgmxy'}[cg]
    pol={'gS':1,'gA':-1,'gT':-1}[cg]
    factor={'gS':1,'gA':1j,'gT':-1j}[cg]

    t_3pt={}
    for tf in tfs:
        insert='_'.join([gm,j,str(tf)])
        t_conn=data[ens]['3pt']['0mom'][ins+'_'+str(tf)]['NJN']
        fla2sgn={'N1':1,'N2':(-1 if j=='j-' else 1)*{'id':1,'g5gz':1,'sgmxy':1}[gm]}
        t_disc=np.mean([(
            data[ens]['3pt']['t;0,0,0;0;{}_t;0,0,0;0;{}'.format(fla,fla)][insert][diag]+
            data[ens]['3pt']['t;0,0,0;1;{}_t;0,0,0;1;{}'.format(fla,fla)][insert][diag]*pol
        )/2*fla2sgn[fla]*factor
                        for fla in flas for diag in diags_3pt],axis=0)
        t_3pt[tf]={'conn':t_conn,'disc':t_disc,'conn+disc':t_conn+t_disc}[case]
        if flags['cc3pt']:
            t_3pt[tf]= (t_3pt[tf] + np.conj(t_3pt[tf][:,::-1])*yu.gtCj[gm]*(np.conj(factor)/factor))/2
        
        t_VEV=data[ens]['VEV']['j']['_'.join([gm,j])]
    
    t=[[t_3pt[tf] for tf in tfs],[t_2pta,t_2ptb],[t_VEV]]
    return t

def run(ins,tfList=None):
    fig, axs = yu.getFigAxs(len(yu.ensembles),3)
    fig.suptitle(ins)
    yu.addRowHeader(axs,[ens[:2]+ens[-2:] for ens in yu.ensembles])
    yu.addColHeader(axs,['conn','conn+disc','disc'])
    
    
    for irow,ens in enumerate(yu.ensembles):
        t={
        'gS_j+':yu.ens2info[ens]['factor_gS'],
        'gS_j-':yu.ens2info[ens]['factor_gS'],
        'gA_j+':yu.ens2info[ens]['factor_gAs'],
        'gA_j-':yu.ens2info[ens]['factor_gAv'],
        'gT_j+':yu.ens2info[ens]['factor_gT'],
        'gT_j-':yu.ens2info[ens]['factor_gT'],
        }
        prefactor=t[ins] if ins in t else 1
        
        tfs=[int(insert.split('_')[-1]) for insert in data[yu.ensembles[0]]['3pt']['0mom'].keys() if insert.startswith(ins)]
        tfs.sort()
        t={'gS_j+':range(8,20+1,2),
        'gS_j-':range(8,18+1,2),
        'gA_j+':range(8,18+1,2),
        'gA_j-':range(8,20+1,2),
        'gT_j+':range(8,16+1,2),
        'gT_j-':range(8,18+1,2),
        }
        tfs=list(t[ins]) if ins in t else tfs  
        if tfList!=None:
            tfs=tfList
        if ens==yu.ensembles[0]:      
            print(tfs)
        
        for icol,case in enumerate(['conn','conn+disc','disc']):
            dat=getdat(ens,ins,tfs,case)
            
            needsVEV=True if case!='conn' and ins in ['gS_j+'] else False
            # needsVEV=False
            
            def func(dat):
                t=yu.meanDeep(dat)
                t=[t[0][i][1:tf]/np.sqrt(
                    np.real(t[1][0][tf])*np.real(t[1][1][tf])*\
                    np.real(t[1][0][1:tf][::-1])/np.real(t[1][0][1:tf])*\
                    np.real(t[1][1][1:tf])/np.real(t[1][1][1:tf][::-1])
                ) - (t[2][0] if needsVEV else 0) 
                for i,tf in enumerate(tfs)]
                # print(t)
                t=yu.prefactorDeep(t,prefactor)
                return t
            
            (mean,err,cov) = yu.jackknife(dat,func)
 
            for i_tf,tf in enumerate(tfs):
                tMean=mean[i_tf];tErr=err[i_tf]
                axs[irow,icol].errorbar(np.arange(1 - tf//2,tf//2),tMean,tErr,\
                    fmt='s',mfc='white')
            dic={
                'gS_j+':([2,11],[-1.5,2]),
                 'gS_j-':([0.3,1.8],[-0.7,0.6]),
                 'gA_j-':([0.95,1.3],[-0.03,0.03]),
                 'gT_j-':([0.9,1.2],[-0.1,0.1])
                 }
            if ins in dic:
                axs[irow,icol].set_ylim(dic[ins][0] if 'conn' in case else dic[ins][1])

In [ ]:
run('gS_j-')

In [ ]:
run('gA_j-')
run('gA_j-',[8,10])

In [ ]:
run('gT_j-')
run('gT_j-',[8])